In [51]:
import pandas as pd
import string

## Load and clean data from Rayyan

In [52]:
df = pd.read_csv("data/articles.csv")
df.head()

,key,title,year,month,day,journal,issn,volume,issue,pages,...,url,language,publisher,location,abstract,notes,doi,keywords,pubmed_id,pmc_id
0,rayyan-354359269,An experience of automated assessment in a lar...,2021.0,1,1,NaN,NaN,29.0,5.0,NaN,...,NaN,NaN,NaN,NaN,The 2020 pandemic imposed new demands on teach...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil...",NaN,NaN,NaN,NaN
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,2021.0,1,1,NaN,NaN,29.0,5.0,NaN,...,NaN,NaN,NaN,NaN,Continuous assessment is essential in educatio...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil...",NaN,NaN,NaN,NaN
2,rayyan-354359271,A tool for evaluating computer programs from s...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Computer science studies are more and more pop...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil...",NaN,Software,NaN,NaN
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,This study proposes an automated programming a...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil...",NaN,NaN,NaN,NaN
4,rayyan-354359273,Analysis of an automatic grading system within...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Reliable and pedagogically sound automated fee...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil...",NaN,NaN,NaN,NaN


### Clean data

In [53]:
df.dropna(axis=1, inplace=True)
df.drop(columns={'month', 'day'}, inplace=True)
df.head()

,key,title,authors,abstract,notes
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"RAYYAN-INCLUSION: {""Marcus""=>""Included"", ""neil..."


In [54]:
df.notes = df.notes.map(lambda input_str: input_str.lstrip('RAYYAN-LABELS: '))

df['screener_1'] = df.notes.map(lambda labels: [label for label in labels.split(',') if ':' not in label])
df['screener_2'] = df.notes.map(lambda labels: [label for label in labels.split(',') if ':' in label])

df.head()

,key,title,authors,abstract,notes,screener_1,screener_2
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[CAE, vpl, mctest, moodle, python, test cases,...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[CAE, java, javassess, test cases, correctness...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[EASEAI, pylint, python, metrics, gamified, co...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[ICKII, test cases, correctness, semi-automati...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[CEP, vpl, java, mule, test cases, correctness...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."


#### Prepend screener 1's labels with topics

In [55]:
def convert_labels(labels):
    output = []
    for label in labels:
        if label in ['vpl', 'mctest', 'moodle', 'javassess', 'pylint', 'coderunner', 'checkstyle', 'sarfgen', 'appgrader', 'better code hub', 'matlab grader',
                     'stepik', 'earsketch', 'pmd', 'pyassessment', 'mule', 'uncode', 'jdbc', 'pass', 'cpplint', 'web-cat', 'cppcheck', 'bluej', 'submitty',
                     'juypter', 'webta', 'docker', 'pyta', 'progedu', 'siette', '2tsw']:
            label = 'tool:' + label
        elif label in ['java', 'python', 'opengl', 'C', 'unknown language', 'sql', 'glsl', 'c++', 'android', 'javascript', 'c#', 'lex', 'yacc',
                       'language  agnostic', 'vb', 'scala', 'matlab', 'ocaml', 'ruby', 'scheme', 'web languages', 'assesmbly', 'typescript', 'prolog']:
            label = 'language:' + label
        elif label in ['correctness', 'code quality', 'oop', 'maintainability', 'graphics', 'complexity', 'documentation', 'concurrency',
                       'teaching context free grammar', 'best practices', 'refactoring', 'distrbuted', 'code design', 'test suite quality', 'productivity']:
            label = 'skill:' + label
        elif label in ['experience report', 'demo paper', 'evaluation paper']:
            label = 'type:' + label
        elif label in ['pre-defined questions', 'test cases', 'pattern matching', 'model solution req', 'dsl for exercise gen', 'mutants',
                       'code repair for feedback', 'metrics', 'program repair', 'static analysis', 'control flow graph', 'model-based', 'unit test expansion',
                       'dynamic analysis', 'ml', 'error prediction', 'fault localisation', 'ci', 'cluster', 'data-driven', 'matching rules', 'language stuctures', 'ci/cd', 'test case generation',
                       'token pattern matching', 'knowledge base', 'reflection', 'hipos', 'graphical output', 'nlp', 'dsl', 'antipatterns', 'mutation testing', 'uml', 'program synthesis']:
            label = 'technique:' + label
        elif label in ['internal exams', 'internal assignments', 'microsoft-dev204.1x', 'internal tutorial', 'leetcode', 'code defenders dataset', 'edX',
                       'github dataset', 'bespoke assignment', 'hackerrank', 'codechef', 'grok', 'dataset available']:
            label = 'data:' + label
        elif label in ['dataset available']:
            label = 'data_available:' + 'True'
        elif label in ['semi-automatic', 'oj']:
            label = 'approach:' + label
        elif label in ['personalised feedback', 'gamified', 'pre-defined feedback', 'plagiarism', 'hints', 'nudge theory', 'visual feedback', 'manual personalised feedback']:
            label = 'feedback:' + label
        elif label in ['compared to human', 'compared to other tools']:
            label = 'evaluation:' + label

        output.append(label)

    return output

Check for outstanding labels to classify

In [56]:
df.screener_1 = df.screener_1.apply(lambda labels: convert_labels(labels))

temp_df = df.screener_1.map(lambda labels: [label for label in labels if ':' not in label and not label.isdigit() and not label[0].isupper()])
temp_df[temp_df.str.len() > 0]

20                          [lms, adaptive learning]
39                                        [accuracy]
48     [item response theory, classical test theory]
56                                [lms, mood design]
62                                       [networked]
64                         [alogthmic understanding]
67                                        [template]
79                              [group contribution]
95                                         [atelier]
99                              [heavily structured]
100                        [operating system grader]
108                                 [error recovery]
Name: screener_1, dtype: object

In [57]:
df.head()

,key,title,authors,abstract,notes,screener_1,screener_2
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[CAE, tool:vpl, tool:mctest, tool:moodle, lang...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[CAE, language:java, tool:javassess, technique...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[EASEAI, tool:pylint, language:python, techniq...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[ICKII, technique:test cases, skill:correctnes...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[CEP, tool:vpl, language:java, tool:mule, tech...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c...."


### Separate out year published and publication location

In [58]:
# Get year from list adapted from: https://stackoverflow.com/questions/363944/python-idiom-to-return-first-item-or-none

df['year_published'] = df.screener_1.apply(lambda labels: next(iter([label for label in labels if label.isdigit()] or []), None))
df.screener_1 = df.screener_1.apply(lambda  labels: [label for label in labels if not label.isdigit()])

df.screener_1 = df.screener_1.map(lambda labels: [label for label in labels if ":" in label])
df.screener_2 = df.screener_2.map(lambda labels: [label for label in labels if ":" in label])

df.head()

,key,title,authors,abstract,notes,screener_1,screener_2,year_published
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[tool:vpl, tool:mctest, tool:moodle, language:...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c....",None
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[language:java, tool:javassess, technique:test...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c....",None
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[tool:pylint, language:python, technique:metri...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c....",None
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[technique:test cases, skill:correctness, appr...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c....",None
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[tool:vpl, language:java, tool:mule, technique...","[INCLUSION: {""Marcus""=>""Included"", ""neil.c.c....",None


#### Screen 2 Data Cleaning

In [59]:
df.screener_2 = df.screener_2.apply(lambda labels: [label.replace('comparison', 'evaluation') for label in labels])

#### Clean tags

In [60]:
punct = string.punctuation.replace(':', '').replace('#', '').replace('+', '') + " "

In [61]:
def clean_tags(labels):
    return [label.translate(str.maketrans(dict.fromkeys(punct, "_"))) for label in labels]

In [62]:
df.screener_1 = df.screener_1.apply(lambda labels: clean_tags(labels))
df.screener_2 = df.screener_2.apply(lambda labels: clean_tags(labels))

df.head()

,key,title,authors,abstract,notes,screener_1,screener_2,year_published
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[tool:vpl, tool:mctest, tool:moodle, language:...","[INCLUSION:___Marcus____Included_, __neil_c_c_...",None
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[language:java, tool:javassess, technique:test...","[INCLUSION:___Marcus____Included_, __neil_c_c_...",None
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[tool:pylint, language:python, technique:metri...","[INCLUSION:___Marcus____Included_, __neil_c_c_...",None
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[technique:test_cases, skill:correctness, appr...","[INCLUSION:___Marcus____Included_, __neil_c_c_...",None
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"INCLUSION: {""Marcus""=>""Included"", ""neil.c.c.br...","[tool:vpl, language:java, tool:mule, technique...","[INCLUSION:___Marcus____Included_, __neil_c_c_...",None


In [63]:
df['tags'] = df.screener_1 + df.screener_2
df.tags = df.tags.apply(lambda tags: list(sorted(set(tags))))

df = df.drop(columns=['screener_1', 'screener_2', 'notes'])

df.head()

,key,title,authors,abstract,year_published,tags
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,None,"[INCLUSION:___Marcus____Included_, __neil_c_c_..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,None,"[INCLUSION:___Marcus____Included_, __neil_c_c_..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,None,"[INCLUSION:___Marcus____Included_, __neil_c_c_..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,None,"[INCLUSION:___Marcus____Included_, __neil_c_c_..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,None,"[INCLUSION:___Marcus____Included_, __neil_c_c_..."


## Validate data

In [64]:
required_prefixes = {'approach', 'language', 'type', 'skill', 'technique', 'data', 'evaluation', 'feedback', 'tool', 'data_available', 'interaction'}

In [65]:
def validate(labels):
    paper_prefixes = []
    for label in labels:
        paper_prefixes.append(label.split(':')[0])

    return sorted([prefix for prefix in required_prefixes if prefix not in paper_prefixes])

validate_df = pd.concat([df.title, df.tags.apply(lambda labels: validate(labels))], axis=1)
validate_df.to_csv('data/validate.csv')

### Example analytics count of all tags

In [66]:
pd.Series([tag for tags in df.tags for tag in tags]).value_counts()

INCLUSION:___Marcus____Included_                                        123
skill:correctness                                                       103
technique:test_cases                                                     72
data:internal_assignments                                                66
technique:unit_testing                                                   47
                                                                       ... 
language:vb                                                               1
feedback:suggest_similar___RAYYAN_EXCLUSION_REASONS:_not_autograding      1
technique:hipos                                                           1
data:internal_tutorial                                                    1
evaluation:student_survey___RAYYAN_EXCLUSION_REASONS:_course_design       1
Length: 201, dtype: int64

## Export